# IMPORT BIBLIOTEK

In [11]:
import mysql.connector
import pandas as pd
import random
import numpy as np

# POŁĄCZENIE Z BAZĄ DANYCH

In [ ]:

con = mysql.connector.connect(
    host = "giniewicz.it",
    user = "team07",
    password = "te@mlot",
    database = "team07"
)

if(con):
    print("Połączenie udane")
else:
    print("Połączenie nieudane")

Połączenie udane


# TWORZENIE SCHEMATU BAZY DANYCH

In [ ]:
cursor = con.cursor()

schema_sql = """
-- Tworzenie tabeli Customers
CREATE TABLE Customers (
  pesel INTEGER PRIMARY KEY,
  first_name VARCHAR(100),
  last_name VARCHAR(100),
  date_of_birth DATE,
  is_insured BOOLEAN,
  email VARCHAR(150),
  phone_number VARCHAR(30),
  emergency_contact_name VARCHAR(100),
  emergency_contact_phone VARCHAR(30),
  emergency_contact_relation VARCHAR(50)
);

-- Tworzenie tabeli Rockets
CREATE TABLE Rockets (
  rocket_id INTEGER PRIMARY KEY,
  name VARCHAR(100),
  max_weight INTEGER,
  manufacturing_date DATE,
  status VARCHAR(50),
  initial_cost DECIMAL(12,2)
);

-- Tworzenie tabeli Spaceships
CREATE TABLE Spaceships (
  spaceship_id INTEGER PRIMARY KEY,
  name VARCHAR(100),
  capacity INTEGER,
  weight INTEGER,
  manufacturing_date DATE,
  status VARCHAR(50),
  initial_cost DECIMAL(12,2)
);

-- Tworzenie tabeli Employees
CREATE TABLE Employees (
  pesel INTEGER PRIMARY KEY,
  first_name VARCHAR(100),
  last_name VARCHAR(100),
  role VARCHAR(100),
  specialization VARCHAR(100),
  assigned_vehicle_id INTEGER,
  is_on_trip BOOLEAN,
  salary DECIMAL(10,2),
  FOREIGN KEY (assigned_vehicle_id) REFERENCES Spaceships(spaceship_id)
);

-- Tworzenie tabeli VehicleApprovals
CREATE TABLE VehicleApprovals (
  approval_id INTEGER PRIMARY KEY,
  vehicle_type VARCHAR(20),
  vehicle_id INTEGER,
  employee_id INTEGER,
  approval_date TIMESTAMP,
  note TEXT,
  FOREIGN KEY (employee_id) REFERENCES Employees(pesel)
);

-- Tworzenie tabeli Destinations
CREATE TABLE Destinations (
  destination_id INTEGER PRIMARY KEY,
  name VARCHAR(100),
  description TEXT,
  distance_range VARCHAR(100)
);

-- Tworzenie tabeli TripTypes
CREATE TABLE TripTypes (
  trip_type_id INTEGER PRIMARY KEY,
  name VARCHAR(100),
  description TEXT,
  estimated_duration_days INTEGER
);

-- Tworzenie tabeli Trips
CREATE TABLE Trips (
  trip_id INTEGER PRIMARY KEY,
  trip_type_id INTEGER,
  destination_id INTEGER,
  launch_date TIMESTAMP,
  return_date TIMESTAMP,
  spaceship_id INTEGER,
  rocket_id INTEGER,
  launch_method VARCHAR(100),
  status VARCHAR(50),
  organization_cost DECIMAL(12,2),
  FOREIGN KEY (trip_type_id) REFERENCES TripTypes(trip_type_id),
  FOREIGN KEY (destination_id) REFERENCES Destinations(destination_id),
  FOREIGN KEY (spaceship_id) REFERENCES Spaceships(spaceship_id),
  FOREIGN KEY (rocket_id) REFERENCES Rockets(rocket_id)
);

-- Tworzenie tabeli TripParticipants
CREATE TABLE TripParticipants (
  participant_id INTEGER PRIMARY KEY,
  trip_id INTEGER,
  customer_id INTEGER,
  ticket_price DECIMAL(10,2),
  status VARCHAR(20),
  FOREIGN KEY (trip_id) REFERENCES Trips(trip_id),
  FOREIGN KEY (customer_id) REFERENCES Customers(pesel)
);

-- Tworzenie tabeli Transactions
CREATE TABLE Transactions (
  transaction_id INTEGER PRIMARY KEY,
  customer_id INTEGER,
  amount DECIMAL(10,2),
  transaction_date TIMESTAMP,
  method VARCHAR(50),
  description TEXT,
  FOREIGN KEY (customer_id) REFERENCES Customers(pesel)
);

-- Tworzenie tabeli TripCrew
CREATE TABLE TripCrew (
  trip_crew_id INTEGER PRIMARY KEY,
  trip_id INTEGER,
  employee_id INTEGER,
  role VARCHAR(50),
  FOREIGN KEY (trip_id) REFERENCES Trips(trip_id),
  FOREIGN KEY (employee_id) REFERENCES Employees(pesel)
);

-- Tworzenie tabeli RecoveryTasks
CREATE TABLE RecoveryTasks (
  recovery_task_id INTEGER PRIMARY KEY,
  rocket_id INTEGER,
  start_date DATE,
  end_date DATE,
  status VARCHAR(50),
  FOREIGN KEY (rocket_id) REFERENCES Rockets(rocket_id)
);

-- Tworzenie tabeli RepairTasks
CREATE TABLE RepairTasks (
  repair_task_id INTEGER PRIMARY KEY,
  spaceship_id INTEGER,
  start_date DATE,
  end_date DATE,
  status VARCHAR(50),
  FOREIGN KEY (spaceship_id) REFERENCES Spaceships(spaceship_id)
);

-- Tworzenie tabeli TaskAssignments
CREATE TABLE TaskAssignments (
  task_assignment_id INTEGER PRIMARY KEY,
  task_type VARCHAR(20),
  task_id INTEGER,
  employee_id INTEGER,
  FOREIGN KEY (employee_id) REFERENCES Employees(pesel)
);

-- Tworzenie tabeli PreFlightChecks
CREATE TABLE PreFlightChecks (
  check_id INTEGER PRIMARY KEY,
  vehicle_type VARCHAR(20),
  vehicle_id INTEGER,
  trip_id INTEGER,
  check_date DATE,
  cost DECIMAL(10,2),
  status VARCHAR(50),
  FOREIGN KEY (trip_id) REFERENCES Trips(trip_id)
);

-- Tworzenie tabeli WaitingList
CREATE TABLE WaitingList (
  waiting_id INTEGER PRIMARY KEY,
  customer_id INTEGER,
  trip_id INTEGER,
  request_date DATE,
  status VARCHAR(50),
  FOREIGN KEY (customer_id) REFERENCES Customers(pesel),
  FOREIGN KEY (trip_id) REFERENCES Trips(trip_id)
);
"""
for statement in schema_sql.strip().split(';'):
    if statement.strip():
        cursor.execute(statement + ';')

con.commit()
print("Schemat bazy danych został utworzony.")

Schemat bazy danych został utworzony.


### DODANIE CELÓW PODRÓŻY

In [7]:
cursor = con.cursor()

schema_sql = """
-- Wstawianie danych do tabeli Destinations
INSERT INTO Destinations (destination_id, name, description, distance_range)
VALUES
  (1, 'Mercury', 'Najbliższa Słońcu planeta, znana z ekstremalnych temperatur i surowego krajobrazu.', '0.61-1.47 AU'),
  (2, 'Venus', 'Gorąca planeta z gęstą atmosferą, często nazywana bliźniaczką Ziemi.', '0.27-1.73 AU'),
  (3, 'Mars', 'Czerwona planeta z potencjalnymi śladami wody i celem przyszłych misji kolonizacyjnych.', '0.52-2.67 AU'),
  (4, 'Moon', 'Naturalny satelita Ziemi, oferujący spektakularne widoki i historyczne miejsca lądowań.', '0.00257 AU'),
  (5, 'Space Station', 'Orbitująca stacja kosmiczna na niskiej orbicie ziemskiej, idealna dla budżetowych wycieczek.', '0.00000267 AU');
"""
for statement in schema_sql.strip().split(';'):
    if statement.strip():
        cursor.execute(statement + ';')

con.commit()
print("Dane do tabeli Destinations zostały wstawione.")

Dane do tabeli Destinations zostały wstawione.


### Dodanie rakiet i statków kosmicznych

In [10]:
cursor = con.cursor()

schema_sql = """
-- Usuwanie istniejących danych z tabel
DELETE FROM Rockets;
DELETE FROM Spaceships;

-- Wstawianie danych do tabeli Rockets
INSERT INTO Rockets (rocket_id, name, max_weight, manufacturing_date, status, initial_cost)
VALUES
  (1, 'StarLifter-I', 30000, '2125-02-10', 'operational', 100000000.00),
  (2, 'StarLifter-II', 40000, '2125-08-15', 'operational', 120000000.00),
  (3, 'StarLifter-III', 50000, '2126-03-20', 'operational', 140000000.00),
  (4, 'OrbitalBoost-I', 60000, '2126-09-05', 'operational', 160000000.00),
  (5, 'OrbitalBoost-II', 70000, '2127-01-12', 'operational', 180000000.00),
  (6, 'MegaFreight-I', 80000, '2127-07-25', 'operational', 200000000.00),
  (7, 'MegaFreight-II', 90000, '2128-02-18', 'operational', 220000000.00),
  (8, 'HeavyStar-I', 100000, '2128-06-30', 'operational', 240000000.00),
  (9, 'HeavyStar-II', 110000, '2129-04-10', 'operational', 260000000.00),
  (10, 'StarLifter-IV', 45000, '2129-11-15', 'operational', 130000000.00),
  (11, 'OrbitalBoost-III', 65000, '2130-03-22', 'operational', 170000000.00),
  (12, 'MegaFreight-III', 95000, '2130-09-10', 'operational', 230000000.00),
  (13, 'HeavyStar-III', 120000, '2131-05-05', 'operational', 280000000.00),
  (14, 'StarLifter-V', 35000, '2131-12-20', 'operational', 110000000.00),
  (15, 'OrbitalBoost-IV', 75000, '2132-07-15', 'operational', 190000000.00),
  (16, 'MegaFreight-IV', 105000, '2133-02-28', 'operational', 250000000.00),
  (17, 'HeavyStar-IV', 130000, '2133-10-10', 'operational', 300000000.00),
  (18, 'StarLifter-VI', 40000, '2134-04-05', 'operational', 125000000.00),
  (19, 'OrbitalBoost-V', 85000, '2135-01-15', 'operational', 210000000.00),
  (20, 'HeavyStar-V', 150000, '2136-06-25', 'operational', 400000000.00);

-- Wstawianie danych do tabeli Spaceships
INSERT INTO Spaceships (spaceship_id, name, capacity, weight, manufacturing_date, status, initial_cost)
VALUES
  (1, 'CosmoVoyager-I', 10, 10000, '2125-03-10', 'operational', 300000000.00),
  (2, 'CosmoVoyager-II', 12, 12000, '2125-09-15', 'operational', 320000000.00),
  (3, 'LunarShuttle-I', 15, 20000, '2126-04-20', 'operational', 400000000.00),
  (4, 'LunarShuttle-II', 18, 22000, '2126-11-05', 'operational', 420000000.00),
  (5, 'OrbitHopper-I', 5, 8000, '2127-02-12', 'operational', 250000000.00),
  (6, 'OrbitHopper-II', 8, 9000, '2127-08-25', 'operational', 270000000.00),
  (7, 'InterplanetaryExpress-I', 30, 40000, '2128-03-18', 'operational', 700000000.00),
  (8, 'InterplanetaryExpress-II', 35, 42000, '2128-09-30', 'operational', 750000000.00),
  (9, 'CosmoVoyager-III', 10, 11000, '2129-05-10', 'operational', 310000000.00),
  (10, 'LunarShuttle-III', 20, 25000, '2130-01-15', 'operational', 450000000.00),
  (11, 'OrbitHopper-III', 6, 8500, '2130-07-22', 'operational', 260000000.00),
  (12, 'InterplanetaryExpress-III', 40, 45000, '2131-04-05', 'operational', 800000000.00),
  (13, 'CosmoVoyager-IV', 12, 13000, '2132-02-28', 'operational', 330000000.00),
  (14, 'LunarShuttle-IV', 25, 30000, '2133-10-10', 'operational', 500000000.00),
  (15, 'InterplanetaryExpress-IV', 50, 50000, '2136-05-25', 'operational', 900000000.00);
"""

try:
    for statement in schema_sql.strip().split(';'):
        if statement.strip():
            cursor.execute(statement)
    con.commit()
    print("Dane do tabel Rockets i Spaceships zostały wstawione.")
except Error as e:
    print(f"Wystąpił błąd MySQL: {e}")
    con.rollback()
finally:
    cursor.close()
    con.close()

Dane do tabel Rockets i Spaceships zostały wstawione.


### Dodanie pracowników

In [ ]:
# Wczytanie danych z plików CSV
imiona_men_df = pd.read_csv('../data/imiona_men.csv')
nazwiska_men_df = pd.read_csv('../data/nazwiska_men.csv')
imiona_woman_df = pd.read_csv('../data/imiona_woman.csv')
nazwiska_woman_df = pd.read_csv('../data/nazwiska_woman.csv')

# Zakładam, że pliki CSV mają kolumny 'imie' i 'nazwisko'
men_names = imiona_men_df['imie'].tolist()
men_surnames = nazwiska_men_df['nazwisko'].tolist()
women_names = imiona_woman_df['imie'].tolist()
women_surnames = nazwiska_woman_df['nazwisko'].tolist()

# Struktura korporacji
employees_structure = [
    {'role': 'CEO', 'specialization': 'executive', 'count': 1, 'salary': 500000.00, 'vehicle': False},
    {'role': 'department_head_service', 'specialization': 'service_management', 'count': 1, 'salary': 200000.00, 'vehicle': False},
    {'role': 'department_head_pilot', 'specialization': 'pilot_management', 'count': 1, 'salary': 200000.00, 'vehicle': False},
    {'role': 'department_head_mechanic', 'specialization': 'mechanic_management', 'count': 1, 'salary': 200000.00, 'vehicle': False},
    {'role': 'department_head_admin', 'specialization': 'admin_management', 'count': 1, 'salary': 200000.00, 'vehicle': False},
    {'role': 'service_staff', 'specialization': 'service', 'count': 100, 'salary': 80000.00, 'vehicle': True},
    {'role': 'pilot', 'specialization': 'pilot', 'count': 20, 'salary': 150000.00, 'vehicle': True},
    {'role': 'mechanic', 'specialization': 'mechanic', 'count': 30, 'salary': 100000.00, 'vehicle': True},
    {'role': 'admin', 'specialization': 'administration', 'count': 40, 'salary': 70000.00, 'vehicle': False},
    {'role': 'space_airport_staff', 'specialization': 'airport_operations', 'count': 1000, 'salary': 50000.00, 'vehicle': False}
]

# Generowanie unikalnych numerów PESEL
pesel_base = 10000000000
pesel_list = np.arange(pesel_base, pesel_base + 1195).tolist()

# Generowanie danych pracowników
employees = []
employee_id = 0

for emp in employees_structure:
    role = emp['role']
    specialization = emp['specialization']
    count = emp['count']
    salary = emp['salary']
    assign_vehicle = emp['vehicle']
    
    for _ in range(count):
        # Losowanie płci (50% mężczyzn, 50% kobiet)
        gender = np.random.choice(['male', 'female'])
        if gender == 'male':
            first_name = np.random.choice(men_names)
            last_name = np.random.choice(men_surnames)
        else:
            first_name = np.random.choice(women_names)
            last_name = np.random.choice(women_surnames)
        
        # Przypisanie pojazdu (tylko dla pilotów, obsługi i mechaników)
        assigned_vehicle_id = np.random.randint(1, 16) if assign_vehicle and np.random.random() > 0.3 else None
        
        employees.append({
            'pesel': pesel_list[employee_id],
            'first_name': first_name,
            'last_name': last_name,
            'role': role,
            'specialization': specialization,
            'assigned_vehicle_id': assigned_vehicle_id,
            'is_on_trip': False,
            'salary': salary
        })
        employee_id += 1

# Generowanie zapytań SQL
sql_statements = [
    "-- Wstawianie danych do tabeli Employees",
    "INSERT INTO Employees (pesel, first_name, last_name, role, specialization, assigned_vehicle_id, is_on_trip, salary) VALUES"
]

values = []
for emp in employees:
    vehicle_id = 'NULL' if emp['assigned_vehicle_id'] is None else emp['assigned_vehicle_id']
    values.append(f"({emp['pesel']}, '{emp['first_name']}', '{emp['last_name']}', '{emp['role']}', '{emp['specialization']}', {vehicle_id}, {emp['is_on_trip']}, {emp['salary']})")

sql_statements.append(',\n'.join(values) + ';')

# Zapis do pliku SQL
with open('Populate_Employees.sql', 'w', encoding='utf-8') as f:
    f.write('\n'.join(sql_statements))

# Wykonanie w MySQL
try:
    con = mysql.connector.connect(
        host="localhost",
        user="your_username",  # Zmień na swoje dane
        password="your_password",  # Zmień na swoje dane
        database="spaceu"  # Zmień na nazwę swojej bazy danych
    )
    cursor = con.cursor()

    # Usunięcie istniejących danych, aby uniknąć konfliktów z kluczem głównym
    cursor.execute("DELETE FROM Employees;")

    # Wykonanie zapytań SQL
    for statement in sql_statements:
        if statement.strip() and not statement.startswith('--'):
            cursor.execute(statement)
    
    con.commit()
    print("Dane do tabeli Employees zostały wstawione.")
except Error as e:
    print(f"Wystąpił błąd MySQL: {e}")
    con.rollback()
finally:
    cursor.close()
    con.close()

In [5]:

# Wczytanie danych z plików CSV
imiona_men_df = pd.read_csv('../data/imiona_men.csv')
nazwiska_men_df = pd.read_csv('../data/nazwiska_men.csv')
imiona_woman_df = pd.read_csv('../data/imiona_woman.csv')
nazwiska_woman_df = pd.read_csv('../data/nazwiska_woman.csv')

# Zamiana na listy imion/nazwisk
imiona_men = imiona_men_df['IMIĘ_PIERWSZE'].dropna().unique().tolist()
nazwiska_men = nazwiska_men_df['Nazwisko aktualne'].dropna().unique().tolist()
imiona_woman = imiona_woman_df['IMIĘ_PIERWSZE'].dropna().unique().tolist()
nazwiska_woman = nazwiska_woman_df['Nazwisko aktualne'].dropna().unique().tolist()

# Losowanie 500 par dla każdej płci
sampled_imiona_men = random.sample(imiona_men, 500)
sampled_nazwiska_men = random.sample(nazwiska_men, 500)
sampled_imiona_woman = random.sample(imiona_woman, 500)
sampled_nazwiska_woman = random.sample(nazwiska_woman, 500)

# Tworzenie listy krotek (first_name, last_name)
pary_men = list(zip(sampled_imiona_men, sampled_nazwiska_men))
pary_woman = list(zip(sampled_imiona_woman, sampled_nazwiska_woman))

# Połączenie obu płci
wszystkie_pary = pary_men + pary_woman
random.shuffle(wszystkie_pary)  # Losowa kolejność

In [24]:
cursor = con.cursor()
sql = "INSERT INTO Customers (first_name, last_name) VALUES (%s, %s)"
cursor.executemany(sql, wszystkie_pary)
con.commit()
print("Wstawiono 1000 klientów.")

Wstawiono 1000 klientów.
